In [ ]:
grid_result = grid_search.fit(X_train, y_train)
fitted_class_mdls[mdl_name+'_reg'] =mldatasets.evaluate_class_mdl(grid_result.best_estimator_,
                    X_train, X_test, y_train, y_test, plot_roc=False,
                    plot_conf_matrix=False, show_summary=False,
                    ret_eval_dict=True)
fitted_class_mdls[mdl_name+'_reg']['cv_results'] =grid_result.cv_results_
fitted_class_mdls[mdl_name+'_reg']['cv_best_params'] =grid_result.best_params_
fitted_class_mdls=
XGBRFClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                colsample_bynode=0.8, colsample_bytree=1, eta=1, gamma=0,
                gpu_id=-1, importance_type='gain', interaction_constraints='',
                learning_rate=1, max_delta_step=0, max_depth=7,
                min_child_weight=1, missing=nan, monotone_constraints='()',
                n_estimators=200, n_jobs=12, num_parallel_tree=200,
                objective='binary:logistic', random_state=23, reg_alpha=0,
                reg_lambda=1e-05, scale_pos_weight=1, seed=23, subsample=0.8,
                tree_method='exact', use_label_encoder=True,
                validate_parameters=1, verbosity=None)
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=7, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=True, random_state=23, verbose=0,
                       warm_start=False)
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eta=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=1, max_delta_step=0, max_depth=7,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=12, num_parallel_tree=1,
              objective='binary:logistic', random_state=23, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=23, subsample=1,
              tree_method='exact', use_label_encoder=True,
              validate_parameters=1, verbosity=None)
LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.7, max_depth=5,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_seed=23, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
               silent=True, subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)

In [ ]:
#Understanding interactions

In [ ]:
fitted_xgbrf_mdl = fitted_class_mdls['xgb-rf_base']['fitted']
# fitted_xgbrf_mdl = fitted_class_mdls['xgb-rf_base']['fitted'][1:-1]
shap_xgbrf_explainer = shap.TreeExplainer(fitted_xgbrf_mdl)

shap_xgbrf_values = shap_xgbrf_explainer.shap_values(X_test)
shap_xgbrf_values_t=shap_xgbrf_explainer.shap_values(X_train)

In [ ]:
plt.figure(figsize=(12,7))

ax0 = plt.subplot(1, 2, 1)
shap.summary_plot(shap_xgbrf_values_t, X_train, plot_type="dot",plot_size=None, show=False)
ax0.set_title("XGBoost SHAP Summary(Train)", fontsize=15)

ax1 = plt.subplot(1, 2, 2)
shap.summary_plot(shap_xgbrf_values, X_test, plot_type="dot",plot_size=None, show=False)
ax1.set_title("XGBoost SHAP Summary(Test)", fontsize=15)

plt.tight_layout()
plt.subplots_adjust(wspace=1)
plt.show()

In [ ]:
shap_xgbrf_interact_values = shap_xgbrf_explainer.shap_interaction_values(X_test)
shap.summary_plot(shap_xgbrf_interact_values, X_test,plot_type="compact_dot", sort=True)

In [ ]:
shap_xgbrf_interact_avgs = np.abs(shap_xgbrf_interact_values).mean(0)
np.fill_diagonal(shap_xgbrf_interact_avgs, 0)
shap_xgbrf_interact_df = pd.DataFrame(shap_xgbrf_interact_avgs)
shap_xgbrf_interact_df.columns = X_test.columns
shap_xgbrf_interact_df.index = X_test.columns

plt.figure(figsize=(40,40))
sns.set(font_scale=1.4)
sns.heatmap(shap_xgbrf_interact_df, cmap='Blues', annot=True,annot_kws={'size':10}, fmt='.2f', linewidths=.2)
plt.show()

In [ ]:
top_features_l = ['comb08'] + ['co2TailpipeGpm', 'fuelType_Diesel',\
                               'co2', 'year', 'ghgScore', 'cylinders']
top_df = fueleconomy_df.loc[X_train.index, top_features_l]

In [ ]:
corrs = stats.spearmanr(top_df).correlation
mask = np.zeros_like(corrs)
mask[np.triu_indices_from(mask)] = True
ax = sns.heatmap(
    corrs, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    xticklabels=top_df.columns,
    yticklabels=top_df.columns,
    mask=mask,
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
)
plt.show()

In [ ]:
print('spearman\tco2TailpipeGpm→comb08\tcorr: %.3f\tp-val: %.4f' % 
      (stats.spearmanr(X_train.co2TailpipeGpm.values, top_df.comb08.values)))
print('point-biserial\tfuelType_Diesel→comb08\tcorr: %.3f\tp-val: %.4f' % 
      (stats.pointbiserialr(top_df.fuelType_Diesel.values, top_df.comb08.values)))
print('spearman\tco2→comb08\t\tcorr: %.3f\tp-val: %.4f' % 
      (stats.spearmanr(X_train.co2.values, top_df.comb08.values)))
print('spearman\tyear→comb08\t\tcorr: %.3f\tp-val: %.4f' % 
      (stats.spearmanr(X_train.year.values, top_df.comb08.values)))
print('spearman\tghgScore→comb08\t\tcorr: %.3f\tp-val: %.4f' % 
      (stats.spearmanr(top_df.ghgScore.values, top_df.comb08.values)))
print('spearman\tcylinders→comb08\tcorr: %.3f\tp-val: %.4f' % 
      (stats.spearmanr(X_train.cylinders.values, top_df.comb08.values)))

In [ ]:
shap.dependence_plot("co2TailpipeGpm", shap_xgb_values_test, X_test,\
                     interaction_index="cylinders", show=False, alpha=0.3)
fig = plt.gcf()
fig.set_size_inches(12,8)
plt.show()
print('spearman\tcylinders→co2TailpipeGpm\tcorr: %.3f\tp-val: %.4f' % 
      (stats.spearmanr(X_train.cylinders.values, X_train.co2TailpipeGpm.values)))
shap.dependence_plot("cylinders", shap_xgb_values_train, X_train,\
                     interaction_index="ghgScore", show=False,\
                     x_jitter=0.4, alpha=0.3)
fig = plt.gcf()
fig.set_size_inches(12,8)
plt.show()
print('spearman\tghgScore→cylinders\t\tcorr: %.3f\tp-val: %.4f' % 
      (stats.spearmanr(top_df.ghgScore.values, top_df.cylinders.values)))

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (13,6))
fig.subplots_adjust(hspace=0, wspace=0.3)
sns.regplot(x=X_train.cylinders, y=X_train.co2TailpipeGpm, ax=axs[0],\
            scatter_kws={'alpha':0.3}, line_kws={'color':'g'})
axs[0].set_ylabel('Tailpipe CO2 in grams/mile', fontsize=13)
axs[0].set_xlabel('Cylinders', fontsize=13)
axs[0].set_xlim([-1,17])
sns.regplot(x=X_train.cylinders, y=y_train, ax=axs[1], marker="+",\
            scatter_kws={'alpha':0.3}, line_kws={'color':'g'})
axs[1].set_ylabel('Combined MPG (comb08)', fontsize=13)
axs[1].set_xlabel('Cylinders', fontsize=13)
axs[1].set_xlim([-1,17])
plt.show() 

In [ ]:
shap.dependence_plot("co2TailpipeGpm", shap_nn_values_test[0],\
                     X_test, alpha=0.3,\
                     interaction_index="co2", show=False)
fig = plt.gcf()
fig.set_size_inches(12,8)
plt.show()
print('spearman\tco2→co2TailpipeGpm\tcorr: %.3f\tp-val: %.4f' % 
      (stats.spearmanr(X_train.co2.values, X_train.co2TailpipeGpm.values)))

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (13,6))
sns.regplot(x=X_train.co2, y=X_train.co2TailpipeGpm, ax=axs[0],\
            scatter_kws={'alpha':0.3}, line_kws={'color':'g'})
axs[0].set_ylabel('Tailpipe CO2 in grams/mile (co2TailpipeGpm)',\
                  fontsize=13)
axs[0].set_xlabel('Tailpipe CO2 in grams/mile (co2)', fontsize=13)
axs[0].set_xlim([-50, 750])
sns.regplot(x=X_train.co2, y=y_train, ax=axs[1], marker="+",\
            scatter_kws={'alpha':0.3}, line_kws={'color':'g'})
axs[1].set_ylabel('Combined MPG (comb08)', fontsize=13)
axs[1].set_xlabel('Tailpipe CO2 in grams/mile (co2)', fontsize=13)
axs[1].set_xlim([-50, 750])
plt.show() 

In [ ]:
shap.dependence_plot("co2", shap_nn_values_test[0],\
                     X_test, alpha=0.3, x_jitter=10,\
                     interaction_index="ghgScore", show=False)
fig = plt.gcf()
fig.set_size_inches(12,8)
plt.show()
print('spearman\tghgScore→co2\tcorr: %.3f\tp-val: %.4f' % 
      (stats.spearmanr(top_df.ghgScore.values, top_df.co2.values)))
shap.dependence_plot("ghgScore", shap_nn_values_test[0],\
                     X_test, alpha=0.3, x_jitter=0.4,\
                     interaction_index="year", show=False)
fig = plt.gcf()
fig.set_size_inches(12,8)
plt.show()
print('spearman\tghgScore→year\tcorr: %.3f\tp-val: %.4f' % 
      (stats.spearmanr(top_df.ghgScore.values, top_df.year.values)))

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (13,6))
sns.regplot(x=X_train.ghgScore, y=X_train.co2TailpipeGpm, ax=axs[0],\
            scatter_kws={'alpha':0.3}, line_kws={'color':'g'})
axs[0].set_ylabel('Tailpipe CO2 in grams/mile (co2TailpipeGpm)',\
                  fontsize=13)
axs[0].set_xlabel('EPA GHG Score (ghgScore)', fontsize=13)
axs[0].set_xlim([-1, 11])
sns.regplot(x=X_train.ghgScore, y=y_train, ax=axs[1], marker="+",\
            scatter_kws={'alpha':0.3}, line_kws={'color':'g'})
axs[1].set_ylabel('Combined MPG (comb08)', fontsize=13)
axs[1].set_xlabel('EPA GHG Score (ghgScore)', fontsize=13)
axs[1].set_xlim([-1, 11])
plt.show() 

In [ ]:
# Force plot

In [ ]:
sample_test_size = 0.05
sample_test_idx = np.random.choice(X_test.shape[0],
                                   math.ceil(X_test.shape[0]*sample_test_size),\
                                   replace=False)
shap.initjs()
shap.force_plot(shap_xgb_explainer.expected_value,\
                shap_xgb_values_test[sample_test_idx],\
                X_test.iloc[sample_test_idx])

In [ ]:
# ALE

In [ ]:
for feature_name in ['co2TailpipeGpm', 'co2', 'ghgScore',\
                     'year', 'cylinders']:
    plt.rc("figure", figsize=(14, 10))
    ale_plot(
        fitted_xgb_model, X_test, [feature_name], bins=10,\
        monte_carlo=True, monte_carlo_rep=50,\
        monte_carlo_ratio=0.4
    )
    plt.show()

In [ ]:
for interaction in [['co2TailpipeGpm', 'co2'], ['co2TailpipeGpm', 'ghgScore'],\
                    ['cylinders', 'co2TailpipeGpm'], ['year', 'co2TailpipeGpm']]:
    plt.rc("figure", figsize=(14, 10))
    ale_plot(
        fitted_xgb_model, X_test, interaction, bins=[10,10]
    )
    plt.show()

In [ ]:
# Global Surrogate

In [ ]:
# Fitting Surrogate

In [ ]:
fitted_dt_surrogate = tree.DecisionTreeRegressor(max_depth=7,\
                                                 random_state=rand).\
                                        fit(X_train, y_train_nn_pred)
y_train_dt_pred = fitted_dt_surrogate.predict(X_train)
y_test_dt_pred = fitted_dt_surrogate.predict(X_test)

In [ ]:
fitted_rf_surrogate = RuleFit(max_rules=150, rfmode='regress',\
                              random_state=rand, tree_size=8).\
                        fit(X_train.astype(float).values,\
                            np.array(y_train_nn_pred).squeeze(),\
                            X_train.columns)
y_train_rf_pred = fitted_rf_surrogate.predict(X_train.astype(float).values)
y_test_rf_pred = fitted_rf_surrogate.predict(X_test.astype(float).values)

In [ ]:
# eval surrogates

In [ ]:
#Measure how well Decision Tree replicates Neural Network's predictions
RMSE_dt_nn_train = metrics.mean_squared_error(y_train_nn_pred,\
                                              y_train_dt_pred,\
                                              squared=False)
RMSE_dt_nn_test = metrics.mean_squared_error(y_test_nn_pred,\
                                             y_test_dt_pred,\
                                             squared=False)
R2_dt_nn_test = metrics.r2_score(y_test_nn_pred, y_test_dt_pred)
#Print all metrics
print('RMSE_train: %.4f\tRMSE_test: %.4f\tr2: %.4f' %\
                (RMSE_dt_nn_train, RMSE_dt_nn_test, R2_dt_nn_test))

In [ ]:
#Measure how well Rule Fit replicates Neural Network's predictions
RMSE_rf_nn_train = metrics.mean_squared_error(y_train_nn_pred,\
                                              y_train_rf_pred,\
                                              squared=False)
RMSE_rf_nn_test = metrics.mean_squared_error(y_test_nn_pred,\
                                             y_test_rf_pred,\
                                             squared=False)
R2_rf_nn_test = metrics.r2_score(y_test_nn_pred, y_test_rf_pred)
#Print all metrics
print('RMSE_train: %.4f\tRMSE_test: %.4f\tr2: %.4f' %\
                (RMSE_rf_nn_train, RMSE_rf_nn_test, R2_rf_nn_test))

In [ ]:
#Interpretting Surrogate

In [ ]:
sns.reset_orig()
fig, axes = plt.subplots(nrows = 1, ncols = 1, figsize = (16,8), dpi=600)
tree.plot_tree(fitted_dt_surrogate,\
               feature_names=X_train.columns.values.tolist(), filled = True, max_depth=2)
fig.show()

In [ ]:
text_tree = tree.export_text(fitted_dt_surrogate,\
               feature_names=X_train.columns.values.tolist())
print(text_tree)

In [ ]:
rulefit_df = fitted_rf_surrogate.get_rules()
rulefit_df = rulefit_df[rulefit_df.coef != 0]
rulefit_df.sort_values(by="importance", ascending=False)

In [ ]:
### Local Interpretation for a group of predictions using Decision Plots

In [ ]:
sample_test_idx = X_test.index.\
                    get_indexer_for([5,6,7,18,19,21,24,25,27])

In [ ]:
expected_value = shap_svm_explainer.expected_value[1]
y_test_shap_pred = (shap_svm_values_test[1].sum(1) + expected_value) > 0.5
print(np.array_equal(y_test_shap_pred, y_test_svc_pred))

In [ ]:
FN = (~y_test_shap_pred[sample_test_idx]) &\
     (y_test.iloc[sample_test_idx] == 1).to_numpy() 

In [ ]:
sns.reset_orig()
plt.rcParams.update(orig_plt_params)
shap.decision_plot(expected_value, shap_svm_values_test[1][sample_test_idx],\
                   X_test.iloc[sample_test_idx], highlight=FN)

In [ ]:
shap.decision_plot(expected_value, shap_svm_values_test[1][696],\
                   X_test.iloc[696], highlight=0)

In [ ]:
### Local Interpretation for a single prediction at a time using Force Plot

In [ ]:
eval_idxs = (X_test.index==5) | (X_test.index==24)
X_test_eval = X_test[eval_idxs]
eval_compare_df = pd.concat([\
            chocolateratings_df.iloc[X_test[eval_idxs].index].rating,\
            pd.DataFrame({'y':y_test[eval_idxs]}, index=[5,24]),\
            pd.DataFrame({'y_pred':y_test_svc_pred[eval_idxs]},\
                         index=[24,5]),
            X_test_eval], axis=1).transpose()
eval_compare_df

In [ ]:
shap.force_plot(expected_value, shap_svm_values_test[1][X_test.index==5],\
                X_test[X_test.index==5],  matplotlib=True)

In [ ]:
shap.force_plot(expected_value, shap_svm_values_test[1][X_test.index==24],\
                X_test[X_test.index==24], matplotlib=True)

In [ ]:
## Employing Local Interpretable Model-agnostic Explanations (LIME)

In [ ]:
### Local Interpretation for a single prediction at a time using LimeTabularExplainer

In [ ]:
lime_svm_explainer = lime.lime_tabular.LimeTabularExplainer(X_test.values,\
                        feature_names=X_test.columns, categorical_features=list(range(3,44)),\
                        class_names=['Not Highly Recomm.', 'Highly Recomm.'])

In [ ]:
lime_svm_explainer.explain_instance(X_test[X_test.index==5].values[0],\
                                    fitted_svm_mdl.predict_proba,\
                                    num_features=8).\
                                show_in_notebook(predict_proba=True)

In [ ]:
lime_svm_explainer.explain_instance(X_test[X_test.index==24].values[0],\
                                    fitted_svm_mdl.predict_proba,\
                                    num_features=8).\
                                show_in_notebook(predict_proba=True)

In [ ]:
#### Getting Acquainted to our “Instance of Interest”

In [ ]:
idx1 = 5231
idx2 = 2726
idx3 = 10127
eval_idxs = X_test.index.isin([idx1, idx2, idx3])
X_test_evals = X_test[eval_idxs]
eval_compare_df = pd.concat([\
            pd.DataFrame({'y':y_test[eval_idxs]},\
                         index=[idx3, idx2, idx1]),\
            pd.DataFrame({'y_pred':y_test_cb_pred[eval_idxs]},\
                         index=[idx3, idx2, idx1]),\
            X_test_evals], axis=1).transpose()
eval_compare_df

In [ ]:
## Understanding Anchor Explanations

In [ ]:
class_names = ['Low Risk', 'Medium/High Risk']

In [ ]:
X_test_eval = np.expand_dims(X_test.values[X_test.\
                                           index.get_loc(idx1)], axis=0)
print(X_test_eval)

In [ ]:
cat_vars_ohe = {5: 2, 7: 6, 13: 8}
print(ohe_to_ord(X_test_eval, cat_vars_ohe)[0])

In [ ]:
category_map = {
    5: ['Female', 'Male'],
    6: ['African-American', 'Asian', 'Caucasian',\
        'Hispanic', 'Native American', 'Other'],
    7: ['Felony 1st Degree', 'Felony 2nd Degree',\
        'Felony 3rd Degree', 'Felony 7th Degree',\
        'Misdemeanor 1st Degree', 'Misdemeanor 2nd Degree',\
        'Misdemeanor 3rd Degree', 'Other Charge Degree']
}
feature_names = ['age', 'juv_fel_count', 'juv_misd_count',\
                 'juv_other_count', 'priors_count', 
                 'sex', 'race', 'c_charge_degree']

In [ ]:
category_map_ohe = {5: ['Not Female', 'Female'], 6: ['Not Male', 'Male'],\
        7:['Not African American', 'African American'],\
        8:['Not Asian', 'Asian'], 9:['Not Caucasian', 'Caucasian'],\
        10:['Not Hispanic', 'Hispanic'],\
        11:['Not Native American', 'Native American'],\
        12:['Not Other Race', 'Other Race'],\
        13:['Not Felony 1st Level', 'Felony 1st Level'],\
        14:['Not Felony 2nd Level', 'Felony 2nd Level'],\
        15:['Not Felony 3rd Level', 'Felony 3rd Level'],\
        16:['Not Felony 7th Level', 'Felony 7th Level'],\
        17:['Not Misdemeanor 1st Deg', 'Misdemeanor 1st Deg'],\
        18:['Not Misdemeanor 2nd Deg', 'Misdemeanor 2nd Deg'],\
        19:['Not Misdemeanor 3rd Deg', 'Misdemeanor 3rd Deg'],\
        20:['Not Other Charge Degree', 'Other Charge Degree']}

In [ ]:
### Local Interpretations for Anchor Explanations

In [ ]:
predict_cb_fn = lambda x: fitted_cb_mdl.predict_proba(x)
anchor_cb_explainer = AnchorTabular(predict_cb_fn, X_train.columns,\
                                    categorical_names=category_map_ohe)
anchor_cb_explainer.fit(X_train.values)

In [ ]:
print('Prediction: %s' %\
      class_names[anchor_cb_explainer.predictor(X_test.loc[idx1].values)[0]])

In [ ]:
anchor_cb_explanation =\
    anchor_cb_explainer.explain(X_test.loc[idx1].values, threshold=0.85,\
                                seed=rand)
print('Anchor: %s' % (' AND\r\n\t'.join(anchor_cb_explanation.anchor)))
print('Precision: %.3f' % anchor_cb_explanation.precision)
print('Coverage: %.3f' % anchor_cb_explanation.coverage)

In [ ]:
anchor_cb_explanation =\
    anchor_cb_explainer.explain(X_test.loc[idx1].values, threshold=0.85,\
                                seed=rand)
print('Anchor: %s' % (' AND\r\n\t'.join(anchor_cb_explanation.anchor)))
print('Precision: %.3f' % anchor_cb_explanation.precision)
print('Coverage: %.3f' % anchor_cb_explanation.coverage)

In [ ]:
anchor_cb_explanation =\
    anchor_cb_explainer.explain(X_test.loc[idx1].values, threshold=0.85,\
                                seed=rand)
print('Anchor: %s' % (' AND\r\n\t'.join(anchor_cb_explanation.anchor)))
print('Precision: %.3f' % anchor_cb_explanation.precision)
print('Coverage: %.3f' % anchor_cb_explanation.coverage)

In [ ]:
predict_nn_fn = lambda x: np.concatenate((1 - fitted_nn_mdl.predict(x),\
                                          fitted_nn_mdl.predict(x)), axis=1)
anchor_nn_explainer = AnchorTabular(predict_nn_fn, X_train.columns,\
                                    categorical_names=category_map_ohe)
anchor_nn_explainer.fit(X_train.values)

In [ ]:
anchor_nn_explanation =\
    anchor_nn_explainer.explain(X_test.loc[idx2].values, threshold=0.85,\
                                seed=rand)
print('Anchor: %s' % (' AND\r\n\t'.join(anchor_nn_explanation.anchor)))
print('Precision: %.3f' % anchor_nn_explanation.precision)
print('Coverage: %.3f' % anchor_nn_explanation.coverage)

In [ ]:
anchor_nn_explanation =\
    anchor_nn_explainer.explain(X_test.loc[idx3].values, threshold=0.85,\
                                seed=rand)
print('Anchor: %s' % (' AND\r\n\t'.join(anchor_nn_explanation.anchor)))
print('Precision: %.3f' % anchor_nn_explanation.precision)
print('Coverage: %.3f' % anchor_nn_explanation.coverage)

In [ ]:
## Exploring Counterfactual Explanations

In [ ]:
### Counterfactual Explanations Guided by Prototypes

In [ ]:
feature_range = (X_train.values.min(axis=0).reshape(1,21).astype(np.float32),\
                 X_train.values.max(axis=0).reshape(1,21).astype(np.float32))
print(feature_range)

In [ ]:
cf_nn_explainer = CounterFactualProto(predict_nn_fn,
                                    X_test_eval.shape,
                                    max_iterations=100,
                                    feature_range=feature_range, 
                                    beta=.1, theta=5,
                                    use_kdtree=True
                                )
cf_nn_explainer.fit(X_test.values, d_type='abdm-mvdm')

In [ ]:
cf_nn_explanation = cf_nn_explainer.explain(X_test_eval)
mldatasets.describe_cf_instance(X_test_eval, cf_nn_explanation, class_names,\
                  cat_vars_ohe, category_map, feature_names)

In [ ]:
### Counterfactual Instances and much more with the What-If-Tool (WIT)

In [ ]:
#### Configuring WIT

In [ ]:
shap_cb_explainer = shap.TreeExplainer(fitted_cb_mdl)

In [ ]:
test_df = recidivism_df.loc[y_test.index]
test_np = test_df.values
cols_l = test_df.columns
delcol_idx = [cols_l.get_loc("is_recid"),\
              cols_l.get_loc("compas_score")]

In [ ]:
def custom_predict_with_shap(examples_np):
    #For shap values we only need same features
    #that were used for training
    inputs_np = np.delete(np.array(examples_np),\
                          delcol_idx, axis=1)

    #Get the model's class predictions
    preds = predict_cb_fn(inputs_np)

    #With test data generate SHAP values which converted
    #to a list of dictionaries format
    keepcols_l = [c for i, c in enumerate(cols_l)\
                  if i not in delcol_idx]
    shap_output = shap_cb_explainer.shap_values(inputs_np)
    attributions = []
    for shap in shap_output:
        attrs = {}
        for i, col in enumerate(keepcols_l):
            attrs[col] = shap[i]
        attributions.append(attrs)
        
    #Prediction function must output 
    #predictions/attributions in dictionary
    output = {'predictions': preds,\
              'attributions': attributions}
    return output

In [ ]:
#### Displaying WIT

In [ ]:
print(y_test.index.get_loc(5231))

In [ ]:
wit_config_builder = WitConfigBuilder(\
                    test_np.tolist(), feature_names=cols_l.tolist()
                ).set_custom_predict_fn(custom_predict_with_shap).\
                set_target_feature("is_recid").set_label_vocab(class_names)
WitWidget(wit_config_builder, height=800)

In [ ]:
## Comparing with the Contrastive Explanation Method (CEM)

In [ ]:
### Train an Autoencoder

In [ ]:
input_layer = tf.keras.Input(shape=(21))
encoder = tf.keras.layers.Dense(10, activation='relu')(input_layer)
bottleneck = tf.keras.layers.Dense(3, activation='relu')(encoder)
decoder = tf.keras.layers.Dense(10, activation='relu')(bottleneck)
output_layer = tf.keras.layers.Dense(21, activation='linear')(decoder)
autoencoder_mdl = tf.keras.Model(input_layer, output_layer)
autoencoder_mdl.summary()

In [ ]:
autoencoder_mdl.compile(loss='mean_squared_error', optimizer='adam')
autoencoder_history = autoencoder_mdl.fit(X_train.values, X_train.values, epochs=16,\
                               batch_size=32, validation_split=0.2, verbose=0)

In [ ]:
### Fit CEM Explainers & Extract Explanations

In [ ]:
cem_nn_explainer_pn = CEM(predict_nn_fn, 'PN',\
                          X_test_eval.shape,\
                          feature_range=feature_range,\
                          max_iterations=100, gamma=100,\
                          ae_model=autoencoder_mdl)
cem_nn_explainer_pn.fit(X_train.values, no_info_type='median')
cem_nn_explanation_pn = cem_nn_explainer_pn.explain(X_test_eval,\
                                                    verbose=False)
print("%s -> %s" % (class_names[cem_nn_explanation_pn.X_pred],\
                    class_names[cem_nn_explanation_pn.PN_pred]))
print("Probabilities: %s" % predict_nn_fn(cem_nn_explanation_pn.PN)[0])
print("Values: %s" % cem_nn_explanation_pn.PN[0])

In [ ]:
cem_nn_explainer_pp = CEM(predict_nn_fn, 'PP',\
                          X_test_eval.shape,\
                          feature_range=feature_range,\
                          max_iterations=100, gamma=100,\
                          ae_model=autoencoder_mdl)
cem_nn_explainer_pp.fit(X_train.values, no_info_type='median')
cem_nn_explanation_pp = cem_nn_explainer_pp.explain(X_test_eval,\
                                                    verbose=False)
print("%s -> %s" % (class_names[cem_nn_explanation_pp.X_pred],\
                    class_names[cem_nn_explanation_pp.PP_pred]))
print("Probabilities: %s" % predict_nn_fn(cem_nn_explanation_pp.PP)[0])
print("Values: %s" % cem_nn_explanation_pp.PP[0])

In [ ]:
### Comparing CEM Explanations

In [ ]:
salients_df = pd.DataFrame({'Feature': X_test.columns,\
                     'x': cem_nn_explanation_pn.X[0],\
                     'PN': cem_nn_explanation_pn.PN[0],\
                     'PN-x': cem_nn_explanation_pn.PN[0]-\
                            cem_nn_explanation_pn.X[0],\
                     'PP': cem_nn_explanation_pp.PP[0]})
salients_df = salients_df[(salients_df.PP != 0) |\
                          (salients_df.PN != 0)]
salients_df

In [ ]:
ebm_mdl = ExplainableBoostingClassifier()
ebm_mdl.fit(X_train, y_train)

In [ ]:
show(ebm_mdl.explain_global())

In [ ]:
ebm_lcl = ebm_mdl.explain_local(X_val,y_val, name='EBM')
show(ebm_lcl)

In [ ]:
ebm_perf = ROC(ebm_mdl.predict_proba).explain_perf(X_val,y_val, name='EBM')
show(ebm_perf)

In [ ]:
X_train_con = X_train.drop(['race'], axis=1).copy()
X_test_con = X_test.drop(['race'], axis=1).copy()

print(fitted_class_mdls['xgb_reg']['cv_best_params'])

best_xgb_params = {'eta': 1.3, 'max_depth': 8, 'reg_alpha': 0.4451,\
                   'reg_lambda': 0.7168, 'scale_pos_weight': 0.9914}
mono_con = (0,0,0,0,0,0,0,0,1)
interact_con = [[4, 5, 6, 7, 8],[0, 1, 2, 3]]

#evaluating constrained model
xgb_con = xgb.XGBClassifier(seed=rand,\
                            monotone_constraints=mono_con,\
                            interaction_constraints=interact_con,\
                            **best_xgb_params)
xgb_con = xgb_con.fit(X_train_con, y_train)
fitted_class_mdls['xgb_con'] =\
        mldatasets.evaluate_class_mdl(xgb_con, X_train_con, X_test_con,\
                                      y_train, y_test, plot_roc=False,\
                                      ret_eval_dict=True)
y_test_pred = fitted_class_mdls['xgb_con']['preds_test']
_ = mldatasets.\
            compare_confusion_matrices(y_test[X_test.race==1], y_test_pred[X_test.race==1],\
                                       y_test[X_test.race==0], y_test_pred[X_test.race==0],\
                                       'Caucasian', 'African-American', compare_fpr=True)

#examining constraints

fitted_xgb_con_mdl = fitted_class_mdls['xgb_con']['fitted']
shap_xgb_con_explainer = shap.TreeExplainer(fitted_xgb_con_mdl)
shap_xgb_con_values = shap_xgb_con_explainer.shap_values(X_test_con)
shap.summary_plot(shap_xgb_con_values, X_test_con, plot_type="dot", show=False)
fig = plt.gcf()
fig.set_size_inches(8,10)
plt.show()

mldatasets.plot_prob_contour_map(recidivism_df.age_group, recidivism_df.priors_per_year,\
                                 recidivism_df.is_recid, x_intervals=ordenc.categories_[2],\
                                 y_intervals=6, use_quartiles=True, xlabel='Age Group',\
                                 ylabel='Priors Per Year', X_df=X_test, x_col='age_group',\
                                 y_col='priors_per_year', model=fitted_xgb_mdl,\
                title='Probability of Recidivism by Age/Priors per Year (according to XGBoost Regularized Model)')

mldatasets.plot_prob_contour_map(recidivism_df.age_group, recidivism_df.priors_per_year,\
                                 recidivism_df.is_recid, x_intervals=ordenc.categories_[2],\
                                 y_intervals=6, use_quartiles=True, xlabel='Age Group',\
                                 ylabel='Priors Per Year', X_df=X_test_con, x_col='age_group',\
                                 y_col='priors_per_year', model=fitted_xgb_con_mdl,\
                title='Probability of Recidivism by Age/Priors per Year (according to XGBoost Constrained Model)')